# Insurance Claim Prediciton

In this notbook:
- loading data
- Feature Engineering

What we can predict in this dataset?
1. __Claim Amount:__ total claims amount per policy holder.
1. __Claim Frequency:__ Number of claims per policy holder per exposure unit `Claim Frequency = Claim Count / Exposure`.
1. __Claim Severity:__ the average claim amount per claim for each policy holder per exposure unit `Claim Severity = Claim Cost / Claim Frequency`.
1. __Avg Claim amount:__ `Avg Claim amount = Claim Amount / Claim Count`
1. __Loss Cost:__ `Loss Cost = Claim Frequency x Claim Severity`
1. __Pure Premium:__ the mean of the total claim amount per exposure unit (the average loss per exposure) `PurePremium  = Claim Amount / Exposure`.

In [13]:
# !conda update scikit-learn -y
!pip install -U scikit-learn

  Using cached scikit_learn-0.24.1-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.1
    Uninstalling scikit-learn-0.22.1:
      Successfully uninstalled scikit-learn-0.22.1


In [1]:
import sklearn
sklearn.__version__ 

'0.24.1'

In [2]:
print(__doc__)

from functools import partial

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import GammaRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_tweedie_deviance
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer

from sklearn.metrics import mean_absolute_error, mean_squared_error, auc

Automatically created module for IPython interactive environment


We have 6780013 individual car insurance policies and for each policy we have 12 variables
1. IDpol: policy number (unique identifier);
2. ClaimNb: number of claims on the given policy;
3. Exposure: total exposure in yearly units;
4. Area: area code (categorical, ordinal);
5. VehPower: power of the car (categorical, ordinal);
6. VehAge: age of the car in years;
7. DrivAge: age of the (most common) driver in years;
8. BonusMalus: bonus-malus level between 50 and 230 (with reference level 100);
9. VehBrand: car brand (categorical, nominal);
10. VehGas: diesel or regular fuel car (binary);
11. Density: density of inhabitants per km2
in the city of the living place of the driver;
12. Region: regions in France (prior to 2016), these are illustrated in Figure 1 (categorical).

In [4]:
df_freq = fetch_openml(data_id=41214, as_frame=True)['data']
df_freq.head(10)

/opt/conda/lib/python3.7/site-packages/sklearn/datasets/_openml.py:854: UserWarning: Version 1 of dataset freMTPL2freq is inactive, meaning that issues have been found in the dataset. Try using a newer version from this URL: https://www.openml.org/data/v1/download/20649148/freMTPL2freq.arff
  data_description['url']))


,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82
1,3.0,1.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82
2,5.0,1.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22
3,10.0,1.0,0.09,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72
4,11.0,1.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72
5,13.0,1.0,0.52,E,6.0,2.0,38.0,50.0,B12,Regular,3003.0,R31
6,15.0,1.0,0.45,E,6.0,2.0,38.0,50.0,B12,Regular,3003.0,R31
7,17.0,1.0,0.27,C,7.0,0.0,33.0,68.0,B12,Diesel,137.0,R91
8,18.0,1.0,0.71,C,7.0,0.0,33.0,68.0,B12,Diesel,137.0,R91
9,21.0,1.0,0.15,B,7.0,0.0,41.0,50.0,B12,Diesel,60.0,R52


In [5]:
df_sev = fetch_openml(data_id=41215, as_frame=True)['data']
df_sev.head()

/opt/conda/lib/python3.7/site-packages/sklearn/datasets/_openml.py:854: UserWarning: Version 1 of dataset freMTPL2sev is inactive, meaning that issues have been found in the dataset. Try using a newer version from this URL: https://www.openml.org/data/v1/download/20649149/freMTPL2sev.arff
  data_description['url']))


,IDpol,ClaimAmount
0,1552.0,995.20
1,1010996.0,1128.12
2,4024277.0,1851.11
3,4007252.0,1204.00
4,4046424.0,1204.00


In [6]:
def load_mtpl2(n_samples=100000):
    """Fetch the French Motor Third-Party Liability Claims dataset.

    Parameters
    ----------
    n_samples: int, default=100000
      number of samples to select (for faster run time). Full dataset has
      678013 samples.
    """
    # freMTPL2freq dataset from https://www.openml.org/d/41214
    df_freq = fetch_openml(data_id=41214, as_frame=True)['data']
    df_freq['IDpol'] = df_freq['IDpol'].astype(np.int)
    df_freq.set_index('IDpol', inplace=True)

    # freMTPL2sev dataset from https://www.openml.org/d/41215
    df_sev = fetch_openml(data_id=41215, as_frame=True)['data']

    # sum ClaimAmount over identical IDs
    df_sev = df_sev.groupby('IDpol').sum()

    df = df_freq.join(df_sev, how="left")
    df["ClaimAmount"].fillna(0, inplace=True)

    # unquote string fields
    for column_name in df.columns[df.dtypes.values == np.object]:
        df[column_name] = df[column_name].str.strip("'")
    return df.iloc[:n_samples]

### Loading datasets, basic feature extraction and target definitions¶
We construct the freMTPL2 dataset by joining the freMTPL2freq table, containing the number of claims (ClaimNb), with the freMTPL2sev table, containing the claim amount (ClaimAmount) for the same policy ids (IDpol).

In [7]:
df = load_mtpl2(n_samples=60000)

# Note: filter out claims with zero amount, as the severity model
# requires strictly positive target values.
df.loc[(df["ClaimAmount"] == 0) & (df["ClaimNb"] >= 1), "ClaimNb"] = 0
df.head()

/opt/conda/lib/python3.7/site-packages/sklearn/datasets/_openml.py:854: UserWarning: Version 1 of dataset freMTPL2freq is inactive, meaning that issues have been found in the dataset. Try using a newer version from this URL: https://www.openml.org/data/v1/download/20649148/freMTPL2freq.arff
  data_description['url']))
/opt/conda/lib/python3.7/site-packages/sklearn/datasets/_openml.py:854: UserWarning: Version 1 of dataset freMTPL2sev is inactive, meaning that issues have been found in the dataset. Try using a newer version from this URL: https://www.openml.org/data/v1/download/20649149/freMTPL2sev.arff
  data_description['url']))


,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
IDpol,,,,,,,,,,,,
1,0.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0
3,0.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0
5,0.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22,0.0
10,0.0,0.09,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0
11,0.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0


In [8]:
# Correct for unreasonable observations (that might be data error)
# and a few exceptionally large claim amounts
df["ClaimNb"] = df["ClaimNb"].clip(upper=4)
df["Exposure"] = df["Exposure"].clip(upper=1)
df["ClaimAmount"] = df["ClaimAmount"].clip(upper=200000)


In [9]:
df.reset_index(inplace=True)

In [10]:
df.shape

(60000, 13)

In [11]:
df.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
0,1,0.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0
1,3,0.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,0.0
2,5,0.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22,0.0
3,10,0.0,0.09,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0
4,11,0.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,0.0


In [12]:
log_scale_transformer = make_pipeline(
    FunctionTransformer(func=np.log),
    StandardScaler()
)

column_trans = ColumnTransformer(
    [
        ("binned_numeric", KBinsDiscretizer(n_bins=3),["VehAge", "DrivAge"]),
        ("onehot_categorical", OneHotEncoder(), ["VehBrand", "VehPower", "VehGas", "Region", "Area"]),
        ("log_scaled_numeric", log_scale_transformer, ["Density"]),
        ("passthrough_numeric", "passthrough",["IDpol","VehAge", "DrivAge","VehBrand", "VehPower", "VehGas", "Region", "Area", "ClaimNb","Exposure", "BonusMalus", "ClaimAmount"]),
    ],
    remainder="drop",
    sparse_threshold=0.0,

)
X = column_trans.fit_transform(df)
X.shape

(60000, 72)

In [13]:
print(X[0,:])

[1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 1.0 0.0 0.0 0.6971986019415635 1 0.0 55.0 'B12' 5.0 'Regular'
 'R82' 'D' 0.0 0.1 50.0 0.0]


In [14]:
bins=[]
for j,f in enumerate(["VehAge","DrivAge"]):
    for idx, val in enumerate(column_trans.transformers_[0][1].bin_edges_[j]):
        edge1= column_trans.transformers_[0][1].bin_edges_[j][idx-1]
        if idx>0:
            bins.append(f+"_bin"+str(edge1)+"_"+str(val))
column_trans.transformers_[1][1].get_feature_names(["VehBrand", "VehPower", "VehGas", "Region", "Area"])
feature_names = bins+\
    column_trans.transformers_[1][1].get_feature_names(["VehBrand", "VehPower", "VehGas", "Region", "Area"]).tolist()+\
    ["Density"]+\
    ["IDpol","VehAge", "DrivAge","VehBrand", "VehPower", "VehGas", "Region", "Area","ClaimNb","Exposure","BonusMalus","ClaimAmount"]


len(feature_names)

72

In [15]:
X.shape

(60000, 72)

In [16]:
feature_names = [x.replace('.','_') for x in feature_names]
feature_names

['VehAge_bin0_0_4_0',
 'VehAge_bin4_0_10_0',
 'VehAge_bin10_0_100_0',
 'DrivAge_bin18_0_36_0',
 'DrivAge_bin36_0_50_0',
 'DrivAge_bin50_0_99_0',
 'VehBrand_B1',
 'VehBrand_B10',
 'VehBrand_B11',
 'VehBrand_B12',
 'VehBrand_B13',
 'VehBrand_B14',
 'VehBrand_B2',
 'VehBrand_B3',
 'VehBrand_B4',
 'VehBrand_B5',
 'VehBrand_B6',
 'VehPower_4_0',
 'VehPower_5_0',
 'VehPower_6_0',
 'VehPower_7_0',
 'VehPower_8_0',
 'VehPower_9_0',
 'VehPower_10_0',
 'VehPower_11_0',
 'VehPower_12_0',
 'VehPower_13_0',
 'VehPower_14_0',
 'VehPower_15_0',
 'VehGas_Diesel',
 'VehGas_Regular',
 'Region_R11',
 'Region_R21',
 'Region_R22',
 'Region_R23',
 'Region_R24',
 'Region_R25',
 'Region_R26',
 'Region_R31',
 'Region_R41',
 'Region_R42',
 'Region_R43',
 'Region_R52',
 'Region_R53',
 'Region_R54',
 'Region_R72',
 'Region_R73',
 'Region_R74',
 'Region_R82',
 'Region_R83',
 'Region_R91',
 'Region_R93',
 'Region_R94',
 'Area_A',
 'Area_B',
 'Area_C',
 'Area_D',
 'Area_E',
 'Area_F',
 'Density',
 'IDpol',
 'VehAge'

In [17]:
df_transformed = pd.DataFrame(data=X, columns= feature_names)
#df_transformed.columns= feature_names
df_transformed.head()

,VehAge_bin0_0_4_0,VehAge_bin4_0_10_0,VehAge_bin10_0_100_0,DrivAge_bin18_0_36_0,DrivAge_bin36_0_50_0,DrivAge_bin50_0_99_0,VehBrand_B1,VehBrand_B10,VehBrand_B11,VehBrand_B12,...,DrivAge,VehBrand,VehPower,VehGas,Region,Area,ClaimNb,Exposure,BonusMalus,ClaimAmount
0,1,0,0,0,0,1,0,0,0,1,...,55,B12,5,Regular,R82,D,0,0.1,50,0
1,1,0,0,0,0,1,0,0,0,1,...,55,B12,5,Regular,R82,D,0,0.77,50,0
2,1,0,0,0,0,1,0,0,0,1,...,52,B12,6,Diesel,R22,B,0,0.75,50,0
3,1,0,0,0,1,0,0,0,0,1,...,46,B12,7,Diesel,R72,B,0,0.09,50,0
4,1,0,0,0,1,0,0,0,0,1,...,46,B12,7,Diesel,R72,B,0,0.84,50,0


Insurances companies are interested in modeling the Pure Premium, that is the expected total claim amount per unit of exposure for each policyholder in their portfolio:

In [18]:
df_transformed["PurePremium"] = df_transformed["ClaimAmount"] / df_transformed["Exposure"]

This can be indirectly approximated by a 2-step modeling: the product of the Frequency times the average claim amount per claim:

In [19]:
df_transformed["Frequency"] = df_transformed["ClaimNb"] / df_transformed["Exposure"]
df_transformed["AvgClaimAmount"] = df_transformed["ClaimAmount"] / np.fmax(df_transformed["ClaimNb"], 1)

In [20]:
df_transformed[df_transformed.ClaimNb>0].head(20)

,VehAge_bin0_0_4_0,VehAge_bin4_0_10_0,VehAge_bin10_0_100_0,DrivAge_bin18_0_36_0,DrivAge_bin36_0_50_0,DrivAge_bin50_0_99_0,VehBrand_B1,VehBrand_B10,VehBrand_B11,VehBrand_B12,...,VehGas,Region,Area,ClaimNb,Exposure,BonusMalus,ClaimAmount,PurePremium,Frequency,AvgClaimAmount
66,1,0,0,0,0,1,0,0,0,1,...,Regular,R11,F,1,0.75,50,303,404,1.33333,303
93,0,1,0,0,0,1,0,0,0,1,...,Diesel,R25,B,1,0.14,60,1981.84,14156,7.14286,1981.84
199,1,0,0,0,1,0,0,0,0,1,...,Regular,R11,E,1,0.14,85,1456.55,10403.9,7.14286,1456.55
205,1,0,0,0,0,1,0,0,0,1,...,Regular,R11,F,2,0.62,100,10834,17474.2,3.22581,5417
223,1,0,0,0,1,0,0,0,0,1,...,Regular,R73,A,1,0.31,50,3986.67,12860.2,3.22581,3986.67
287,0,1,0,0,0,1,0,0,0,1,...,Diesel,R93,D,1,0.84,50,1840.14,2190.64,1.19048,1840.14
295,1,0,0,1,0,0,0,0,0,1,...,Regular,R31,D,1,0.75,64,1397.97,1863.96,1.33333,1397.97
388,1,0,0,0,1,0,0,0,0,1,...,Regular,R93,E,1,0.76,50,971.98,1278.92,1.31579,971.98
396,0,0,1,1,0,0,1,0,0,0,...,Regular,R11,E,1,0.68,105,1442.75,2121.69,1.47059,1442.75
468,1,0,0,0,0,1,0,0,0,1,...,Regular,R93,D,1,0.73,50,637.41,873.164,1.36986,637.41


In [22]:
df_transformed.to_csv('dataset_after_FE.csv', index = False)